In [1]:
import base64
import json
import pandas as pd
import requests
import tensorflow as tf

In [15]:
df = pd.read_csv("hf://datasets/sakren/twitter_racism_dataset/twitter_racism_parsed_dataset.csv")

df.sample(10)

,index,id,Text,Annotation,oh_label
132,5.691813e+17,5.691813e+17,i don't know because i had everyone else on th...,none,0
3936,5.736065e+17,5.736065e+17,"Yes, I'm aware of GGs efforts with the IRS. No...",none,0
12839,5.691785e+17,5.691785e+17,@1Life0Continues a number of variables. :) i'...,none,0
7690,5.525246e+17,5.525246e+17,"RT @riwired: .@ORehaf Oh No.Do NOT go to ""Isla...",racism,1
11226,5.633276e+17,5.633276e+17,No. http://t.co/ljHm8Wd27r,none,0
2885,5.758901e+17,5.758901e+17,@glitchDreams @Popehat doesn't work. I don't t...,none,0
8892,5.755054e+17,5.755054e+17,@SAIACarleton @Naazzk What about the environme...,racism,1
11006,5.755982e+17,5.755982e+17,RT @norme73: #mkr i noticed Kat and Andre were...,none,0
7814,5.723310e+17,5.723310e+17,RT @benpobjie: “We see a future for ourselves ...,none,0
7023,5.723321e+17,5.723321e+17,"Lynn, I would have scored them a 2 #MKR",none,0


In [17]:
testing_text = df["Text"][132]
testing_text

"i don't know because i had everyone else on that thread blocked. but i'm am loling so hard right now."

In [18]:
def json_prepare(text):
    feature_spec = {
        "text": tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])
        )
    }

    example = tf.train.Example(
        features = tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{"examples": {"b64": base64.b64encode(example).decode()}}]

    return json.dumps({"signature_name": "serving_default", "instances": result})

In [19]:
def predict(text):
    json_data = json_prepare(text)

    endpoint = "http://localhost:8080/v1/models/racism-detection-model:predict"
    response = requests.post(endpoint, data=json_data)
    print(response.json())
    print(json_data)
    prediction = response.json()["predictions"][132][0]

    return prediction

In [20]:
predict(testing_text)

{'error': 'Name: <unknown>, Feature: Annotation (data type: string) is required but could not be found.\n\t [[{{function_node __inference_serve_tf_examples_fn_293400}}{{node ParseExample/ParseExampleV2}}]]'}
{"signature_name": "serving_default", "instances": [{"examples": {"b64": "CnMKcQoEdGV4dBJpCmcKZWkgZG9uJ3Qga25vdyBiZWNhdXNlIGkgaGFkIGV2ZXJ5b25lIGVsc2Ugb24gdGhhdCB0aHJlYWQgYmxvY2tlZC4gYnV0IGknbSBhbSBsb2xpbmcgc28gaGFyZCByaWdodCBub3cu"}}]}


KeyError: 'predictions'